In [11]:
import numpy as np
import matplotlib.pyplot as plt
import struct
import math
from statistics import mean, stdev

import qiskit
from qiskit import *

import pennylane as qml
from pennylane import numpy as pnp

import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

In [15]:
iterations = 2
img_size = 10
binary_MSE = []
amplitude_MSE = []

for i in tqdm(range(0, iterations)):
    
    initial, visibilities = generate_img(img_size) 
    
    # BINARY
    binary_readout = binary(visibilities)
    
    # AMPLITUDE
    n_n = img_size*img_size
    inp = visibilities.flatten()
    norm = qml.math.sum(qml.math.abs(inp) ** 2)
    number_of_qbits = math.ceil(math.log2(n_n))
    wires = range(number_of_qbits)
    amp_dev = qml.device('default.qubit', wires)
    @qml.qnode(amp_dev)
    def amp_encoding(data):
        qml.AmplitudeEmbedding(data, wires, normalize=True, pad_with=0.)
        return qml.state()
    readout = amp_encoding(inp).astype('complex64')
    readout.numpy()
    readout = readout*qml.math.sqrt(norm) 
    readout = readout[:n_n] 
    readout = np.array(readout).reshape(img_size , img_size)
    amplitude_readout = np.fft.ifft2(readout).astype('complex64')
    
    binary_mse = MSE(initial, binary_readout)
    amplitude_mse = MSE(initial, amplitude_readout)
    
    binary_MSE.append(binary_mse)
    amplitude_MSE.append(amplitude_mse)

print('Binary MSEs: '+ str(binary_MSE))
print()
print('Amplitude MSEs: '+ str(amplitude_MSE))
print()

binary_mean = mean(binary_MSE)
print('Binary mean: '+ '\033[1m' + str(binary_mean) + '\033[0m')
binary_std = np.std(binary_MSE)
print('Binary std: '+ '\033[1m' + str(binary_std) + '\033[0m')
amplitude_mean = mean(amplitude_MSE)
print('Amplitude mean: '+ '\033[1m' + str(amplitude_mean) + '\033[0m')
amplitude_std = np.std(amplitude_MSE)
print('Amplitude std: '+ '\033[1m' + str(amplitude_std) + '\033[0m')

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.60s/it]

Binary MSEs: [5.503049e-14, 3.83518e-14]

Amplitude MSEs: [7.929514e-13, 5.4626937e-14]

Binary mean: 4.6691146e-14
Binary std: 8.339346e-15
Amplitude mean: 4.2378918e-13
Amplitude std: 3.6916225e-13


In [3]:
# float to binary 
def float_to_bin_real(num):
    return format(struct.unpack('!I', struct.pack('!f', num))[0], '032b')
def float_to_bin_imag(num):
    return format(struct.unpack('!I', struct.pack('!f', num))[0], '032b')

# binary to float
def bin_to_float_real(binary):
    return struct.unpack('!f',struct.pack('!I', int(binary, 2)))[0]
def bin_to_float_imag(binary):
    return struct.unpack('!f',struct.pack('!I', int(binary, 2)))[0]

# reverse a string (for the measurement step readout)
def reverse(string):
    string = string[::-1]
    return string

In [4]:
def generate_img(n):

    sky_image = np.zeros((n, n), dtype='complex64')
    mask = np.zeros((n, n), dtype='bool')
    stars = np.random.randint(1, 7) 
    radius = 1 

    for i in range(0, stars):
        index_i = np.random.randint(0, n-radius)
        index_j = np.random.randint(0, n-radius)
        for j in range(0, radius):
            for k in range(0, radius):
                sky_image[index_i+j][index_j+k] = complex(np.random.rand()*100, np.random.rand()*100)
                mask[index_i+j][index_j+k] = True
            
    sky_image.real += np.random.randn(n,n)/100
    sky_image.imag += np.random.randn(n,n)/100
    
    visibilities = np.fft.fft2(sky_image).astype('complex64')
    
    return sky_image, visibilities

In [5]:
def binary_encode(qc, binary, off_set):
    
    for i in range(0, len(binary)):
        qc.reset(off_set+i) 

        if binary[i]=='1':
            qc.x(off_set+i) 
    
    off_set += len(binary)

    return off_set

In [6]:
def binary(img):
    
    number_of_pixels = img.shape[0]*img.shape[1]
    number_of_bits_real = 32
    number_of_bits_imag = 32
    number_of_bits = number_of_bits_real + number_of_bits_imag
    number_of_qbits = number_of_pixels*number_of_bits
    off_set = 0

    qc = QuantumCircuit(number_of_qbits)
    for i in range(0, img.shape[0]):
        for j in range(0, img.shape[1]):
                binary_real = float_to_bin_real(img[i, j].real)
                binary_imag = float_to_bin_imag(img[i, j].imag)
                binary = binary_real+binary_imag
                off_set = binary_encode(qc, binary, off_set)
                
    qc.measure_all()
    backend = Aer.get_backend('aer_simulator') 
    job = backend.run(qc, shots=1, memory=True) 
    output = job.result().get_memory()[0]
    out = reverse(output)
    
    chunks_real = []
    chunks_imag = []
    for i in range(0, number_of_pixels):
        chunks_real.append(out[number_of_bits*i:(number_of_bits*i)+number_of_bits_real]) 
        chunks_imag.append(out[(number_of_bits*i)+number_of_bits_imag:(number_of_bits*i)+number_of_bits_imag+number_of_bits_imag]) 

    readout = []
    for i in range(0, len(chunks_real)):
        readout.append(complex(bin_to_float_real(chunks_real[i]), bin_to_float_imag(chunks_imag[i]))) 

    readout = np.array(readout).reshape(img.shape[0] , img.shape[1]).astype('complex64')
    readout = np.fft.ifft2(readout).astype('complex64') #IFT
    
    return readout

In [8]:
def MSE(img1, img2):
    diff_real = (img1.real - img2.real)
    diff_imag = (img1.imag - img2.imag)
    diff_sum = diff_real + diff_imag
    
    mse = (diff_sum**2).mean()
    
    return mse